# 제 1고지 미분 자동 계산

###### 1단계 상자로서의 변수
###### 2단계 변수를 낳는 함수
###### 3단계 함수 연결
###### 4단계 수치 미분
###### 5단계 역전파 이론
###### 6단계 수동 역전파 이론
###### 7단계 역전파 자동화
###### 8단계 재귀에서 반복문으로
###### 9단계 함수를 더 편리하게
###### 10단계 테스트

In [5]:
import numpy as np
import matplotlib as plt

## step 1 상자로서의 변수

In [3]:
# Variable 클래스 구현

class Variable :
    def __init__(self, data) :
        self.data = data

In [3]:


data = np.array(1.0)
x = Variable(data)

In [4]:
print(x.data)

1.0


In [5]:
x.data = np.array(2.0)

In [6]:
print(x.data)

2.0


## step2 변수를 낳는 함수

In [8]:
# Function 클래스 구현
class Function :
    def __call__(self, input) :
        x = input.data
        y = x**2
        output = Variable(y)
        return output

In [9]:
x = Variable(np.array(10))
f = Function()
y = f(x)

In [12]:
print(y)

In [13]:
print(y.data)

100


In [15]:
# real implementation

class Function :
    
    def __call__(self, input) :
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output
        
    def forward(self, x) :
        raise NotImplementedError()

In [16]:
# square implementation

class Square(Function) :
    def forward(self, x) :
        return x ** 2

In [18]:
x = Variable(np.array(100))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
10000


## step 3 함수 연결

In [19]:
## exp 함수 구현

class Exp(Function) :
    def forward(self, x) :
        return np.exp(x)
    

In [20]:
A = Square()
B = Exp()
C = Square()

In [24]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## step4 수치 미분